In [29]:
import pandas as pd
import numpy as np
import pickle
import time

In [30]:
label_list = [0, 1, 2, 3, 4]

In [31]:
# choose individuum
subject = "m292"

In [32]:
hidden_dim = 10

In [33]:
# Load persistence diagrams

persistence_diagrams  = np.load('Embeddings_and_Persistence_Diagrams/'+str(subject)+'/Persistence_Diagrams_All_Labels_with_Hidden_Dim_'+str(hidden_dim)+'.npy', \
    allow_pickle=True).item() # .item() to convert the dtype to dict again

In [34]:
# TODO do this in Preprocessing_And_Computing_...

reshaped_persistence_diagrams = {}

for label in label_list:
    reshaped_persistence_diagrams["Label_"+str(label)] = [persistence_diagram[0] for persistence_diagram in list(persistence_diagrams["Label_"+str(label)])]

persistence_diagrams = reshaped_persistence_diagrams

In [35]:
# Adcock-Carlson coordinates

In [36]:
def carlsson_coordinates(persistence_diagrams):
    n = len(persistence_diagrams)
    persistence_diagrams_features_cc1 = np.zeros(n)
    persistence_diagrams_features_cc2 = np.zeros(n)
    persistence_diagrams_features_cc3 = np.zeros(n)
    persistence_diagrams_features_cc4 = np.zeros(n)
    start = time.time()
    ymax = 0
    for i in range(0,n):
        if len(persistence_diagrams[i])>0:
            b = np.max(persistence_diagrams[i][:,1])
        else:
            b = ymax
        if ymax < b:
            ymax = b
        else:
            ymax = ymax
    for i in range(0,n):
        if len(persistence_diagrams[i])>0:
            x = persistence_diagrams[i][:,0]
            y = persistence_diagrams[i][:,1]
            persistence_diagrams_features_cc1[i] = sum(x*(y-x))
            persistence_diagrams_features_cc2[i] = sum((ymax - y)*(y-x))
            persistence_diagrams_features_cc3[i] = sum(x**2*(y-x)**4)
            persistence_diagrams_features_cc4[i] = sum((ymax-y)**2*(y-x)**4)
        else:
            persistence_diagrams_features_cc1[i] = 0
            persistence_diagrams_features_cc2[i] = 0
            persistence_diagrams_features_cc3[i] = 0
            persistence_diagrams_features_cc4[i] = 0
            
    return persistence_diagrams_features_cc1, persistence_diagrams_features_cc2, persistence_diagrams_features_cc3, persistence_diagrams_features_cc4

In [37]:
def create_dataframe(hidden_dim, X_features_cc1, X_features_cc2, X_features_cc3, X_features_cc4, label, train):
    feature_df = pd.DataFrame()

    feature_df["Joint_hidden_dim_"+str(hidden_dim)+"_AC_coordinate_cc1"] = X_features_cc1
    feature_df["Joint_hidden_dim_"+str(hidden_dim)+"_AC_coordinate_cc2"] = X_features_cc2
    feature_df["Joint_hidden_dim_"+str(hidden_dim)+"_AC_coordinate_cc3"] = X_features_cc3
    feature_df["Joint_hidden_dim_"+str(hidden_dim)+"_AC_coordinate_cc4"] = X_features_cc4
    feature_df["Label"] = label

    # Subject
    feature_df["Subject"] = subject

    # Training or test set (boolean)
    feature_df["Train"] = train

    return feature_df

In [38]:
features_dataframes = {}


for label in label_list:
    
    # Train data Adcock Carlson coordinates 
    coord_1, coord_2, coord_3, coord_4 = carlsson_coordinates(persistence_diagrams["Label_"+str(label)])
    features_dataframes[label] = create_dataframe(hidden_dim, coord_1, coord_2, coord_3, coord_4, \
                                                        label, True)

In [39]:
# Concatenate and save dataframes
feature_df = pd.concat([features_dataframes[0], features_dataframes[1], features_dataframes[2], \
                              features_dataframes[3], features_dataframes[4]], ignore_index=True)

feature_df.to_csv("Features/"+str(subject)+"/Advanced_Features_Hidden_Dim_"+str(hidden_dim)+".csv")